In [233]:
import recordlinkage as rl
import pandas as pd

In [234]:
schema_path = "./csv/schema_final.csv"

In [235]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True)

# Preprocessamento

In [236]:
from recordlinkage.preprocessing import *

## Cleaning dei dati ...

In [237]:
schema_cleaned = schema

In [238]:
for i in range(len(schema.index)):
    row = schema.iloc[i]
    row_series = row.squeeze()
    nome = pd.Series(row["name"])
    try:
        nome_cleaned = clean(nome, lowercase=True, replace_by_none='', replace_by_whitespace='', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
    except:
        print(nome)
    row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$]+', replace_by_whitespace=r'[\\-\\_]', strip_accents=None, remove_brackets=True, encoding='utf-8', decode_error='strict')
    schema_cleaned.iloc[i] = row_series_cleaned
    schema_cleaned.iloc[i]["name"] = nome_cleaned[0]
print(schema_cleaned)

# Occorrenze token per riga

In [ ]:
# value_occurence(schema)

# Blocking

In [ ]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned, schema_cleaned)
print(len(candidate_links))

1286933


In [ ]:
print(len(candidate_links)/len(schema_cleaned)**2)

3.619683663640771e-05


# Pre-Labeling

In [ ]:
['name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'Stock', 'share price', 'city', 'address', 'website']

['name',
 'country',
 'market cap',
 'founded year',
 'employees',
 'industry',
 'sector',
 'ceo',
 'revenue',
 'Stock',
 'share price',
 'city',
 'address',
 'website']

In [ ]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [ ]:
features

name   country  founded year  industry  sector  address  city  \
0      0        1.0  1.000000          1.00  1.000000     0.0      0.0   0.0   
       2966     1.0  0.230769          0.75  0.000000     0.0      0.0   0.0   
       12966    1.0  0.000000          0.25  0.000000     0.0      0.0   0.0   
       30538    1.0  1.000000          0.75  0.173913     0.0      0.0   0.0   
       34398    1.0  0.230769          0.00  0.000000     0.0      0.0   0.0   
...             ...       ...           ...       ...     ...      ...   ...   
180541 180541   1.0  0.000000          0.00  1.000000     0.0      1.0   0.0   
180542 180542   1.0  0.000000          0.00  1.000000     0.0      1.0   0.0   
180543 180543   1.0  0.000000          0.00  1.000000     0.0      1.0   0.0   
180544 180544   1.0  0.000000          0.00  1.000000     0.0      1.0   0.0   
180545 180545   1.0  0.000000          0.00  1.000000     0.0      1.0   0.0   

                    ceo  
0      0       1.000000  
       2966    0.666667  
       12966   0.480000  
       30538   0.571429  
       34398   0.000000  
...                 ...  
180541 180541  0.000000  
180542 180542  0.000000  
180543 180543  0.000000  
180544 180544  0.000000  
180545 180545  0.000000  

[1286933 rows x 8 columns]

In [ ]:
features.describe()

,name,country,founded year,industry,sector,address,city,ceo
count,1286933.0,1.286933e+06,1.286933e+06,1.286933e+06,1.286933e+06,1.286933e+06,1.286933e+06,1.286933e+06
mean,1.0,4.423471e-01,7.132380e-02,5.948174e-02,5.024128e-02,2.044610e-02,9.935383e-03,7.112032e-02
std,0.0,4.906092e-01,2.517902e-01,2.336935e-01,2.180986e-01,1.302606e-01,9.916248e-02,2.477167e-01
min,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.000000      1728
5.972222         2
5.966102         2
5.961538         2
5.956522         2
             ...  
1.023810         2
1.021898         4
1.019231         6
1.017857         2
1.000000    535487
Length: 3360, dtype: int64

In [ ]:
features = features[features.sum(axis=1) > 3]

name  country  founded year  industry  sector  address  city  \
0      0        1.0      1.0      1.000000  1.000000     0.0      0.0   0.0   
       30538    1.0      1.0      0.750000  0.173913     0.0      0.0   0.0   
2966   2966     1.0      1.0      1.000000  0.000000     0.0      0.0   0.0   
       178886   1.0      1.0      0.666667  0.000000     0.0      0.0   0.0   
30538  0        1.0      1.0      0.750000  0.173913     0.0      0.0   0.0   
...             ...      ...           ...       ...     ...      ...   ...   
177024 177024   1.0      1.0      1.000000  1.000000     0.0      0.0   0.0   
177025 177025   1.0      1.0      1.000000  1.000000     0.0      0.0   0.0   
177026 177026   1.0      1.0      1.000000  1.000000     0.0      0.0   0.0   
177027 177027   1.0      1.0      1.000000  1.000000     0.0      0.0   0.0   
177028 177028   1.0      1.0      1.000000  1.000000     0.0      0.0   0.0   

                    ceo  
0      0       1.000000  
       30538   0.571429  
2966   2966    1.000000  
       178886  1.000000  
30538  0       0.571429  
...                 ...  
177024 177024  0.000000  
177025 177025  0.000000  
177026 177026  0.000000  
177027 177027  0.000000  
177028 177028  0.000000  

[87358 rows x 8 columns]

In [ ]:
len(features)

1286933

In [ ]:
for i in range(len(features)):
    multiIdxs = features.iloc[i].name
    if multiIdxs[0] == multiIdxs[1]:
        features.drop(multiIdxs)

features


Found diagonal (0, 0)
Found diagonal (2966, 2966)
Found diagonal (12966, 12966)
Found diagonal (30538, 30538)
Found diagonal (34398, 34398)
Found diagonal (47265, 47265)
Found diagonal (54942, 54942)
Found diagonal (65661, 65661)
Found diagonal (78817, 78817)
Found diagonal (79872, 79872)
Found diagonal (86721, 86721)
Found diagonal (97407, 97407)
Found diagonal (115881, 115881)
Found diagonal (129166, 129166)
Found diagonal (132604, 132604)
Found diagonal (133611, 133611)
Found diagonal (144284, 144284)
Found diagonal (160732, 160732)
Found diagonal (163832, 163832)
Found diagonal (177036, 177036)
Found diagonal (178886, 178886)
Found diagonal (180557, 180557)
Found diagonal (181399, 181399)
Found diagonal (1, 1)
Found diagonal (2995, 2995)
Found diagonal (12995, 12995)
Found diagonal (29574, 29574)
Found diagonal (34434, 34434)
Found diagonal (47222, 47222)
Found diagonal (54970, 54970)
Found diagonal (65632, 65632)
Found diagonal (71737, 71737)
Found diagonal (79833, 79833)
Found di

KeyboardInterrupt: 